# Merging

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from gerrychain import Graph, Partition, Election, MarkovChain
from gerrychain.updaters import Tally, cut_edges
from gerrychain.proposals import propose_random_flip
from gerrychain.accept import always_accept

In [2]:
PRECINCT_FOLDER = 'ga_2020_gen_2020_blocks' #2020 presidential election results by block; only want precinctid.
CENSUS_FOLDER = 'ga_ds248_2020_block' #2020 census data by block. Merge to get demographics for precincts. 
BOUNDARY_FOLDER = 'ga_2020' #2020 presidential election results by precinct. Want precinct name and boundaries. 
ELECTION_FILE = '2020-HOUSE_precinct_general.csv' #2020 house election results. Use to link precincts with districts. 
PROPOSED_PLAN = 'CONGRESS-PROP1-2021-shape' #Proposed plan. 
STATE_FP = '13' #Use for election file. 
PRECINCT_COLUMNS = ['GEOID20','PRECINCTID']
CENSUS_COLUMNS = ['GEOID20','U7E001','U7E002','U7E005','U7E006']
BOUNDARY_COLUMNS = ['PRECINCTID', 'PRECINCT_NAME', 'geometry', 'CTYNAME'] 
EXPORT_PRECINCTS = 'ga_precinct_2020.csv'
EXPORT_DISTRICTS = 'ga_districts_2020.csv'
EXPORT_EDGES = 'ga_edges_2020.csv'
DISTRICTS = 14

METER_LENGTH = 60.96 #200 feet

In [3]:
precinct_data = gpd.read_file(PRECINCT_FOLDER + '/' + PRECINCT_FOLDER + '.shp')
census_data = gpd.read_file(CENSUS_FOLDER + '/' + CENSUS_FOLDER + '.csv')
boundary_data = gpd.read_file(BOUNDARY_FOLDER + '/' + BOUNDARY_FOLDER + '.shp')
election_data = gpd.read_file(ELECTION_FILE) 
proposed_plan = gpd.read_file(PROPOSED_PLAN + '/' + PROPOSED_PLAN + '.shp')

In [4]:
precinct_data.head()

,GEOID20,STATEFP,COUNTYFP,PRECINCTID,VAP_MOD,G20PRERTRU,G20PREDBID,G20PRELJOR,G20USSRPER,G20USSDOSS,G20USSLHAZ,G20PSCRSHA,G20PSCDBRY,G20PSCLMEL,G20PSCRMCD,G20PSCDBLA,G20PSCLWIL,geometry
0,130019501001000,13,001,0011B,12,7.29,0.97,0.06,6.93,1.12,0.13,6.82,0.91,0.20,6.64,0.96,0.18,"POLYGON ((-82.34985 31.92087, -82.34960 31.920..."
1,130019501001001,13,001,0011B,9,5.46,0.73,0.04,5.20,0.84,0.10,5.11,0.68,0.15,4.98,0.72,0.13,"POLYGON ((-82.33439 31.90390, -82.33416 31.904..."
2,130019501001002,13,001,0011B,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"POLYGON ((-82.35306 31.93902, -82.34760 31.938..."
3,130019501001003,13,001,0011B,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"POLYGON ((-82.31536 31.94550, -82.31516 31.945..."
4,130019501001004,13,001,0011B,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"POLYGON ((-82.35489 31.93956, -82.35484 31.939..."


In [5]:
census_data.head()

,GISJOIN,YEAR,GEOID,GEOCODE,REGIONA,DIVISIONA,STATE,STATEA,COUNTY,COUNTYA,...,U7E065,U7E066,U7E067,U7E068,U7E069,U7E070,U7E071,U7E072,U7E073,geometry
0,G13000109501001000,2020,1010000US130019501001000,1.3002E+14,3,5,Georgia,13,Appling County,1,...,0,0,0,0,0,0,0,0,0,None
1,G13000109501001001,2020,1010000US130019501001001,1.3002E+14,3,5,Georgia,13,Appling County,1,...,0,0,0,0,0,0,0,0,0,None
2,G13000109501001002,2020,1010000US130019501001002,1.3002E+14,3,5,Georgia,13,Appling County,1,...,0,0,0,0,0,0,0,0,0,None
3,G13000109501001003,2020,1010000US130019501001003,1.3002E+14,3,5,Georgia,13,Appling County,1,...,0,0,0,0,0,0,0,0,0,None
4,G13000109501001004,2020,1010000US130019501001004,1.3002E+14,3,5,Georgia,13,Appling County,1,...,0,0,0,0,0,0,0,0,0,None


In [6]:
boundary_data.head()

,DISTRICT,CTYSOSID,PRECINCT_I,PRECINCT_N,CTYNAME,CTYNUMBER,CTYNUMBER2,FIPS2,G20PRERTRU,G20PREDBID,...,G20PSCRMCD,G20PSCDBLA,G20PSCLWIL,R21USSRPER,R21USSDOSS,R21USSRLOE,R21USSDWAR,R21PSCRMCD,R21PSCDBLA,geometry
0,215122,215122,122,FIRST AFRICAN,MUSCOGEE,106,106,215,238,668,...,251,587,44,230,589,222,599,239,564,"POLYGON ((-84.96984 32.46725, -84.97031 32.467..."
1,215108,215108,108,ST MARK/HEIFERHORN,MUSCOGEE,106,106,215,3243,1676,...,3268,1456,122,3071,1484,3055,1499,3112,1397,"POLYGON ((-84.96552 32.53259, -84.96852 32.532..."
2,057031,057031,031,R T JONES,CHEROKEE,28,028,057,1021,513,...,998,461,60,891,455,879,467,902,434,"POLYGON ((-84.46579 34.25122, -84.46545 34.251..."
3,057033,057033,033,SALACOA,CHEROKEE,28,028,057,454,69,...,419,61,17,419,69,421,68,416,64,"POLYGON ((-84.53036 34.38103, -84.53047 34.380..."
4,01506,01506,06,CENTER,BARTOW,8,008,015,2312,568,...,2230,516,107,2026,507,2019,511,2032,478,"MULTIPOLYGON (((-84.65788 34.14247, -84.65830 ..."


In [7]:
election_data = election_data[election_data['state_fips'] == STATE_FP] #Filter by state. 
election_data.head()

,precinct,office,party_detailed,party_simplified,mode,votes,county_name,county_fips,jurisdiction_name,jurisdiction_fips,...,special,writein,state_po,state_fips,state_cen,state_ic,date,readme_check,magnitude,geometry
336237,100,US HOUSE,DEMOCRAT,DEMOCRAT,ADVANCED VOTING,873,WARE,13299,WARE,13299,...,FALSE,FALSE,GA,13,58,44,2020-11-03,FALSE,1,None
336238,100,US HOUSE,DEMOCRAT,DEMOCRAT,ELECTION DAY,216,WARE,13299,WARE,13299,...,FALSE,FALSE,GA,13,58,44,2020-11-03,FALSE,1,None
336239,1-01C FIRST PRESBYTERIAN CHURCH,US HOUSE,DEMOCRAT,DEMOCRAT,ABSENTEE,613,CHATHAM,13051,CHATHAM,13051,...,FALSE,FALSE,GA,13,58,44,2020-11-03,FALSE,1,None
336240,1-01C FIRST PRESBYTERIAN CHURCH,US HOUSE,DEMOCRAT,DEMOCRAT,ADVANCED VOTING,635,CHATHAM,13051,CHATHAM,13051,...,FALSE,FALSE,GA,13,58,44,2020-11-03,FALSE,1,None
336241,1-01C FIRST PRESBYTERIAN CHURCH,US HOUSE,DEMOCRAT,DEMOCRAT,ELECTION DAY,175,CHATHAM,13051,CHATHAM,13051,...,FALSE,FALSE,GA,13,58,44,2020-11-03,FALSE,1,None


In [8]:
#Get Democrat and Republican Votes
election_data['votes'] = election_data['votes'].astype('int64')
precinct_votes = election_data.groupby(['precinct', 'party_detailed', 'district', 'county_name']).sum(numeric_only = True).reset_index()

In [9]:
dem_votes = precinct_votes[precinct_votes['party_detailed'] == 'DEMOCRAT'][['precinct', 'district', 'votes', 'county_name']].reset_index()
rep_votes = precinct_votes[precinct_votes['party_detailed'] == 'REPUBLICAN'][['votes']].reset_index()

In [10]:
election_data = dem_votes.join(rep_votes, lsuffix = '_Dem', rsuffix = '_Rep').drop(columns = ['index_Dem', 'index_Rep'])
election_data[['votes_Dem', 'votes_Rep']] = election_data[['votes_Dem', 'votes_Rep']].astype('int64')
election_data['Total_Votes'] = election_data['votes_Dem'] + election_data['votes_Rep']

In [11]:
proposed_plan.head()

,ID,AREA,DATA,DISTRICT,MEMBERS,LOCKED,NAME,POPULATION,F18_POP,NH_WHT,...,F_AP_HWN,F_AP_OTH,F_18_AP_WH,F_18_AP_IN,F_18_AP_AS,F_18_AP_HW,F_18_AP_OT,F_PEOPLE_I,IDEAL_VALU,geometry
0,1,599.114258,5,013,1.0,None,None,765137,574789,125106,...,0.001472,0.104958,0.000416,0.000029,0.000043,0.000001,0.000143,0.006238,765136.0,"POLYGON ((-84.49513 33.80854, -84.49533 33.808..."
1,2,1168.543210,3,011,1.0,None,None,765137,595201,469264,...,0.001412,0.115147,0.000668,0.000038,0.000068,0.000003,0.000138,0.015703,765136.0,"POLYGON ((-84.45549 33.82681, -84.45554 33.826..."
2,3,250.457443,10,005,1.0,None,None,765137,621515,273819,...,0.001432,0.064441,0.000271,0.000013,0.000033,0.000001,0.000036,0.050784,765136.0,"POLYGON ((-84.50359 33.79871, -84.50263 33.799..."
3,257,417.795990,9,004,1.0,None,None,765135,589470,197536,...,0.001445,0.104404,0.000381,0.000018,0.000037,0.000001,0.000060,0.008533,765136.0,"POLYGON ((-83.83929 33.55701, -83.83932 33.556..."
4,258,322.790497,12,007,1.0,None,None,765137,566934,225905,...,0.001771,0.205277,0.000138,0.000008,0.000042,0.000000,0.000045,0.002537,765136.0,"POLYGON ((-83.84318 33.89478, -83.84529 33.893..."


In [12]:
census_data['GEOID20'] = census_data['GEOID'].apply(lambda x: x.split('US')[1]) #Acquiring GEOIDs 
census_data['Other'] = census_data.apply(lambda x: int(x['U7E001']) - int(x['U7E002']) - int(x['U7E005']) - int(x['U7E006']), axis = 1)
boundary_data = boundary_data.rename(columns = {'DISTRICT': 'PRECINCTID', 'PRECINCT_N': 'PRECINCT_NAME'}) 
election_data = election_data.rename(columns = {'precinct': 'PRECINCT_NAME', 'county_name' : 'CTYNAME'})

In [13]:
precinct_data = precinct_data[PRECINCT_COLUMNS]
census_data = census_data[CENSUS_COLUMNS]
boundary_data = boundary_data[BOUNDARY_COLUMNS]

In [14]:
precinct_data = precinct_data.merge(census_data, on = 'GEOID20') #Merge election + census first
precinct_data[CENSUS_COLUMNS] = precinct_data[CENSUS_COLUMNS].astype('int64') #Listed as Object by default.
precinct_data = precinct_data.drop(columns = 'GEOID20')
precinct_data = precinct_data.groupby(by='PRECINCTID')
precinct_data = precinct_data.sum() 
precinct_data = precinct_data.merge(boundary_data, on = 'PRECINCTID') #Merge with boundaries.
precinct_data = precinct_data.merge(election_data, on = ['PRECINCT_NAME', 'CTYNAME'], how = 'left') #Merge with elections. 
precinct_data = precinct_data.rename(columns = {'PRECINCTID': 'Precinct_ID',
                                                'PRECINCT_NAME': 'Precinct_Name',
                                                'votes_Dem': 'Democrat',
                                                'votes_Rep': 'Republican',
                                                'U7E001': 'Total_Population', 
                                                'U7E002': 'Hispanic', 
                                                'U7E005': 'White', 
                                                'U7E006': 'Black',
                                                'district': 'District'})
precinct_data = gpd.GeoDataFrame(precinct_data, geometry = 'geometry')
precinct_data = precinct_data.drop(columns = 'CTYNAME')
precinct_data.head()

,Precinct_ID,Total_Population,Hispanic,White,Black,Precinct_Name,geometry,District,Democrat,Republican,Total_Votes
0,0011B,1433,46,1267,72,1B,"POLYGON ((-82.31535 31.94637, -82.31508 31.946...",012,126.0,834.0,960.0
1,0011C,1192,65,991,117,1C,"POLYGON ((-82.43146 31.87040, -82.43131 31.870...",012,41.0,581.0,622.0
2,0012,2639,264,955,1360,2,"POLYGON ((-82.36296 31.78329, -82.36294 31.783...",012,859.0,522.0,1381.0
3,0013A1,865,12,817,17,3A1,"POLYGON ((-82.28323 31.90526, -82.28321 31.905...",012,19.0,572.0,591.0
4,0013C,1920,144,1247,457,3C,"POLYGON ((-82.35010 31.77878, -82.34979 31.778...",012,271.0,823.0,1094.0


In [15]:
#Clean overlapping precincts. 
drop_index = [] 
for i in precinct_data[precinct_data['Precinct_ID'].duplicated(keep = False)]['Precinct_ID'].unique():
    index = precinct_data[precinct_data['Precinct_ID'] == i].index
    drop_index.append(precinct_data.loc[index, 'Total_Votes'].idxmin()) 

In [16]:
#Groupby
temp = precinct_data[['Precinct_ID', 'Democrat', 'Republican', 'Total_Votes']] #Only add these columns. 
temp = temp.groupby('Precinct_ID').sum()
precinct_data = temp.merge(precinct_data, on = ['Precinct_ID'])
precinct_data = precinct_data.drop(drop_index)
precinct_data = precinct_data.drop(columns = ['Democrat_y', 'Republican_y', 'Total_Votes_y'])
precinct_data = precinct_data.rename(columns = {'Democrat_x': 'Democrat', 'Republican_x': 'Republican', 'Total_Votes_x': 'Total_Votes'})

In [17]:
precinct_data = gpd.GeoDataFrame(precinct_data, geometry = 'geometry')

# Adjacency

In [36]:
precinct_data = precinct_data.to_crs("EPSG:4326")
precinct_data['geometry'] = precinct_data.buffer(0)

In [19]:
def getIntersection(geometry):
    intersection = precinct_data.intersection(geometry)
    intersection = intersection.to_crs("EPSG:3857")
    lengths = intersection[~intersection.is_empty].length
    lengths = lengths[lengths > METER_LENGTH]
    return precinct_data.loc[lengths.index, 'Precinct_ID'].values

In [37]:
precinct_data['Neighbors'] = precinct_data.apply(lambda x: getIntersection(x['geometry']), axis = 1)

# Postprocessing

In [15]:
#precinct_data['District'] = np.random.randint(1, DISTRICTS, precinct_data.shape[0])
#precinct_data['Other'] = precinct_data.apply(lambda x: x['Total_Population'] - x['Hispanic'] - x['White'] - x['Black'], axis = 1)

In [16]:
#precinct_data.groupby('District').sum(numeric_only = True)

,Republican,Democrat,Total_Population,Hispanic,White,Black,Other
District,,,,,,,
1,160067,189823,589700,52545,300200,188671,48284
2,159445,179512,544227,55840,264867,182956,40564
3,208304,207686,691004,58291,379231,196470,57012
4,186385,192987,644296,50169,340105,204850,49172
5,230984,204049,705241,67401,403056,179630,55154
6,161266,167394,561079,59844,289099,170599,41537
7,145843,157177,500791,39941,263200,155716,41934
8,222647,221083,735544,59323,387815,231225,57181
9,177536,174751,591142,52871,316150,172762,49359


# MGGG

In [13]:
#precinct_data['geometry'] = gpd.GeoSeries.buffer(precinct_data['geometry'], distance = 0)
#precinct_data['District'] = np.random.randint(1, DISTRICTS, precinct_data.shape[0])

In [14]:
#graph = Graph().from_geodataframe(precinct_data)

c:\users\tungd.laptop-8m0b2cr6\appdata\local\programs\python\python39\lib\site-packages\gerrychain\graph\adjacency.py:21: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  geometries[i].id = i
c:\users\tungd.laptop-8m0b2cr6\appdata\local\programs\python\python39\lib\site-packages\gerrychain\graph\adjacency.py:23: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  tree = STRtree(geometries)
c:\users\tungd.laptop-8m0b2cr6\appdata\local\programs\python\python39\lib\site-packages\gerrychain\graph\adjacency.py:66: UserWarning: Found overlaps among the given polygons. Indices of overlaps: {(48, 1766), (1676, 1828), (570, 1552), (1152, 1163), (1366, 1480), (1377, 1510), (1388, 1510), (1023, 1406), (2038, 2049), (2226, 2228), (539, 1293), (1405, 1480), (1200, 1209), (1341, 1343), (621, 1273), (1185, 1196), (1551, 1552), (1092, 1404), (1005, 1366), (12

In [15]:
#election = Election('PRED20', {'Dem': 'Democrat', 'Rep': 'Republican'})

In [16]:
#initial_partition = Partition(
#    graph,
#    assignment="District",
#    updaters={
#        "cut_edges": cut_edges,
#        "population": Tally("Total_Population", alias="population"),
#        "PRED20": election
#    }
#)

In [17]:
#chain = MarkovChain(
#    proposal=propose_random_flip,
#    constraints=[],
#    accept=always_accept,
#    initial_state=initial_partition,
#    total_steps=10000
#)

In [18]:
#for partition in chain:
#    print(sorted(partition["PRED20"].percents("Dem")))

[0.44901075968840154, 0.47955922110810717, 0.48338816236643917, 0.49311841195291184, 0.5008780842900257, 0.5022286101870095, 0.5048342388903614, 0.5051588070311381, 0.5116037928917389, 0.5130589021610598, 0.5178352282067045, 0.5212196321541407, 0.5516310616896599]
[0.44901075968840154, 0.47955922110810717, 0.48338816236643917, 0.49381631943475096, 0.5008780842900257, 0.5022286101870095, 0.5048342388903614, 0.5051588070311381, 0.5116037928917389, 0.5130589021610598, 0.5171765721492465, 0.5212196321541407, 0.5516310616896599]
[0.44901075968840154, 0.47955922110810717, 0.4838116709512958, 0.49347289132032335, 0.5008780842900257, 0.5022286101870095, 0.5048342388903614, 0.5051588070311381, 0.5116037928917389, 0.5130589021610598, 0.5171765721492465, 0.5212196321541407, 0.5516310616896599]
[0.4510348705664613, 0.47955922110810717, 0.4816198323671998, 0.49347289132032335, 0.5008780842900257, 0.5022286101870095, 0.5048342388903614, 0.5051588070311381, 0.5116037928917389, 0.5130589021610598, 0.5

[0.4627370994829914, 0.47122088728268796, 0.47835064735683946, 0.488647415631104, 0.4907570147148326, 0.4916657681344536, 0.5082877885635043, 0.5133890342962472, 0.5168695174068427, 0.5197079995473761, 0.528081846295235, 0.5328220658402144, 0.5338890993359775]
[0.4627370994829914, 0.47122088728268796, 0.47835064735683946, 0.488647415631104, 0.4907570147148326, 0.4916657681344536, 0.5080187414133784, 0.513596328774767, 0.5168695174068427, 0.5197079995473761, 0.528081846295235, 0.5328220658402144, 0.5338890993359775]
[0.46375861475550895, 0.47122088728268796, 0.47835064735683946, 0.488647415631104, 0.4907570147148326, 0.4916657681344536, 0.5080187414133784, 0.513596328774767, 0.5168695174068427, 0.5197079995473761, 0.527146754795878, 0.5328220658402144, 0.5338890993359775]
[0.46375861475550895, 0.47122088728268796, 0.47835064735683946, 0.488647415631104, 0.4907570147148326, 0.4916657681344536, 0.5059744075803305, 0.513596328774767, 0.5168695174068427, 0.5197079995473761, 0.52714675479587

[0.4583353375578166, 0.4622127800440402, 0.4721958046492527, 0.4850108328959325, 0.49116388683192874, 0.5026523449522682, 0.5031072922167825, 0.5080415264477736, 0.5130846166483934, 0.527915143696696, 0.5311380999132708, 0.5319023094780004, 0.5466348496257253]
[0.4583353375578166, 0.4622127800440402, 0.47349063282349785, 0.4850108328959325, 0.4896869358099903, 0.5026523449522682, 0.5031072922167825, 0.5080415264477736, 0.5130846166483934, 0.527915143696696, 0.5311380999132708, 0.5319023094780004, 0.5466348496257253]
[0.4583353375578166, 0.4622127800440402, 0.47349063282349785, 0.4850108328959325, 0.4896869358099903, 0.5027898438307995, 0.5031072922167825, 0.5080415264477736, 0.5130846166483934, 0.527915143696696, 0.5311380999132708, 0.5319023094780004, 0.5465023220007145]
[0.4583353375578166, 0.4622127800440402, 0.47349063282349785, 0.4850108328959325, 0.4896869358099903, 0.5027898438307995, 0.5031072922167825, 0.5080415264477736, 0.5130846166483934, 0.527915143696696, 0.53095409640921

[0.4476034072932745, 0.4666241079028351, 0.47214557780442823, 0.4815015771710242, 0.49762957869017294, 0.4981614061239879, 0.5088141821432118, 0.5092904441525008, 0.5175796741328698, 0.5257020427071385, 0.5288098929067965, 0.5318268121515093, 0.5432488973047748]
[0.4476034072932745, 0.4666241079028351, 0.47214557780442823, 0.4815015771710242, 0.4972004275426958, 0.4981614061239879, 0.5088141821432118, 0.5092904441525008, 0.5181000914827238, 0.5257020427071385, 0.5288098929067965, 0.5318268121515093, 0.5432488973047748]
[0.4476034072932745, 0.4666241079028351, 0.47214557780442823, 0.4815015771710242, 0.4972004275426958, 0.4981614061239879, 0.5088141821432118, 0.5092904441525008, 0.5168261078040395, 0.5257020427071385, 0.5298031080702098, 0.5318268121515093, 0.5432488973047748]
[0.44367231763701476, 0.4666241079028351, 0.47214557780442823, 0.4815015771710242, 0.4972004275426958, 0.5016635391363411, 0.5088141821432118, 0.5092904441525008, 0.5168261078040395, 0.5257020427071385, 0.52980310

[0.4470527643411261, 0.4624939270307131, 0.465205690912618, 0.48316218326621796, 0.4911640617156682, 0.499570706505103, 0.4998602910245427, 0.5007437762846866, 0.5028375462304813, 0.5283626101418063, 0.5320530585212567, 0.540712443140421, 0.5780654657470865]
[0.4470527643411261, 0.4624939270307131, 0.465205690912618, 0.48316218326621796, 0.4911640617156682, 0.499570706505103, 0.4998602910245427, 0.5007437762846866, 0.5028375462304813, 0.5277460724245981, 0.5320530585212567, 0.5413741816993611, 0.5780654657470865]
[0.4470527643411261, 0.4624939270307131, 0.465205690912618, 0.48316218326621796, 0.4903069764698075, 0.499570706505103, 0.4998602910245427, 0.5007437762846866, 0.5028375462304813, 0.5277460724245981, 0.5327207121195103, 0.5413741816993611, 0.5780654657470865]
[0.4470527643411261, 0.4624939270307131, 0.465205690912618, 0.4866525255079345, 0.4866704326410878, 0.499570706505103, 0.4998602910245427, 0.5007437762846866, 0.5028375462304813, 0.5277460724245981, 0.5327207121195103, 0.

...

[0.4520913729060978, 0.4714958078288154, 0.47472597401316113, 0.47534366568949205, 0.47600890495105724, 0.4826075279618167, 0.4845425798950308, 0.5070483829079346, 0.5091274026511051, 0.5333751420844276, 0.5371674305100886, 0.5415728095354555, 0.5809712265899515]
[0.4520913729060978, 0.4714958078288154, 0.47341453294673086, 0.47600890495105724, 0.4765995983408097, 0.4826075279618167, 0.4845425798950308, 0.5070483829079346, 0.5091274026511051, 0.5333751420844276, 0.5371674305100886, 0.5415728095354555, 0.5809712265899515]
[0.4520913729060978, 0.4714958078288154, 0.47341453294673086, 0.47376665087137365, 0.4765995983408097, 0.4826075279618167, 0.4869452805539046, 0.5070483829079346, 0.5091274026511051, 0.5333751420844276, 0.5371674305100886, 0.5415728095354555, 0.5809712265899515]
[0.4520913729060978, 0.4714958078288154, 0.47341453294673086, 0.4739231179305285, 0.4765995983408097, 0.4826075279618167, 0.4869452805539046, 0.5070483829079346, 0.5091274026511051, 0.5333751420844276, 0.537167

[0.4480612374824923, 0.46476051720997996, 0.47829873929703987, 0.48494719080057497, 0.4890890731895486, 0.49165735976419034, 0.49166499749624437, 0.49222776092086884, 0.49488614877492715, 0.507788125072222, 0.5473146616270934, 0.5637869462806777, 0.5734185724642457]
[0.4480612374824923, 0.46476051720997996, 0.47829873929703987, 0.48494719080057497, 0.4890890731895486, 0.49165735976419034, 0.49166499749624437, 0.49222776092086884, 0.4940568475452196, 0.507788125072222, 0.54840887080207, 0.5637869462806777, 0.5734185724642457]
[0.4480612374824923, 0.46476051720997996, 0.47829873929703987, 0.48494719080057497, 0.4890890731895486, 0.4914472989750833, 0.49165735976419034, 0.49222776092086884, 0.4940568475452196, 0.507788125072222, 0.54840887080207, 0.5637212685481524, 0.5734185724642457]
[0.4480612374824923, 0.46476051720997996, 0.47829873929703987, 0.4855895414400479, 0.4890890731895486, 0.4914472989750833, 0.49165735976419034, 0.49222776092086884, 0.4935079464315093, 0.507788125072222, 0.

[0.44684237696455203, 0.4512040184637421, 0.4707858844535878, 0.47658068052148755, 0.477856007267772, 0.4883420514218595, 0.4939317930032435, 0.5000762404318259, 0.5015358080675949, 0.5117657149564071, 0.5656251958036053, 0.5719313332528353, 0.5929216360083552]
[0.44684237696455203, 0.4514267732331589, 0.4707858844535878, 0.47658068052148755, 0.477856007267772, 0.4883420514218595, 0.4936033051253393, 0.5000762404318259, 0.5015358080675949, 0.5117657149564071, 0.5656251958036053, 0.5719313332528353, 0.5929216360083552]
[0.44684237696455203, 0.4514267732331589, 0.4707858844535878, 0.47476416267012606, 0.477856007267772, 0.4883420514218595, 0.4936033051253393, 0.5000762404318259, 0.5033065886305332, 0.5117657149564071, 0.5656251958036053, 0.5719313332528353, 0.5929216360083552]
[0.44684237696455203, 0.4514267732331589, 0.46942053239263654, 0.47476416267012606, 0.477856007267772, 0.4883420514218595, 0.4936033051253393, 0.5000762404318259, 0.5033065886305332, 0.5117657149564071, 0.565625195

[0.4322747908387959, 0.44877861261576096, 0.46743038648199803, 0.4718352573202758, 0.4882968820475418, 0.501410867726554, 0.5014837319001255, 0.5040194106238136, 0.5119110282506477, 0.5142890738911622, 0.5527619880483895, 0.5786981578239384, 0.5917772371211306]
[0.4322747908387959, 0.44877861261576096, 0.46743038648199803, 0.4718352573202758, 0.4882968820475418, 0.49976063533284804, 0.501410867726554, 0.5040194106238136, 0.5119110282506477, 0.5142890738911622, 0.5527619880483895, 0.5818895223027604, 0.5917772371211306]
[0.4322747908387959, 0.45016734627915095, 0.46743038648199803, 0.4718352573202758, 0.4882968820475418, 0.49967452599037965, 0.49976063533284804, 0.5040194106238136, 0.5119110282506477, 0.5142890738911622, 0.5527619880483895, 0.5818895223027604, 0.5917772371211306]
[0.4322747908387959, 0.45016734627915095, 0.46743038648199803, 0.4718352573202758, 0.4882968820475418, 0.49976063533284804, 0.5002981383459676, 0.5040194106238136, 0.5119110282506477, 0.5136572145899081, 0.5527

[0.4066436594583246, 0.42296629932235835, 0.47111336078963717, 0.4910241576753129, 0.49541667106694404, 0.4989480425526868, 0.5058956728958947, 0.5089656184717165, 0.5170852876815781, 0.5279431738792764, 0.556538557752735, 0.5799057068103515, 0.5936071745782886]
[0.4066436594583246, 0.42296629932235835, 0.47111336078963717, 0.49221670064559975, 0.4939196591121047, 0.4989480425526868, 0.5058956728958947, 0.5089656184717165, 0.5170852876815781, 0.5279431738792764, 0.556538557752735, 0.5799057068103515, 0.5936071745782886]
[0.40622568093385214, 0.42332623643816564, 0.47111336078963717, 0.49221670064559975, 0.4939196591121047, 0.4989480425526868, 0.5058956728958947, 0.5089656184717165, 0.5170852876815781, 0.5279431738792764, 0.556538557752735, 0.5799057068103515, 0.5936071745782886]
[0.40622568093385214, 0.42332623643816564, 0.47111336078963717, 0.49221670064559975, 0.4939196591121047, 0.4989480425526868, 0.5058956728958947, 0.5089656184717165, 0.5170852876815781, 0.5279431738792764, 0.556

# Exports

In [39]:
precinct_data.to_csv(EXPORT_PRECINCTS)